In [ ]:
import pandas as pd
canaritos = 5
df = pd.read_csv('~/datasets/competencia_02_target.csv')
clientes = df[df["foto_mes"] == 202108]["numero_de_cliente"].unique()
df = df[df["numero_de_cliente"].isin(clientes)]

features_to_drop = ["cprestamos_prendarios", "mprestamos_prendarios", "cprestamos_personales", "mprestamos_personales", "clase_ternaria"]
df = df.drop(columns=features_to_drop)
df

In [ ]:
from dmeyf2025.processors.feature_processors import CleanZerosTransformer, DeltaLagTransformer, PercentileTransformer, PeriodStatsTransformer, TendencyTransformer, IntraMonthTransformer, RandomForestFeaturesTransformer, DatesTransformer, HistoricalFeaturesTransformer, AddCanaritos
from experiment_utils import *
def get_features(X, training_months):
    X_transformed = X
    
    X_transformed = apply_transformer(
        CleanZerosTransformer(),
        X_transformed,
        "CleanZerosTransformer",
        logger
    )
    X_transformed = apply_transformer(
        HistoricalFeaturesTransformer(),
        X_transformed,
        "HistoricalFeaturesTransformer",
        logger
    )
    initial_cols =X_transformed.columns
    X_transformed = apply_transformer(
        PeriodStatsTransformer(
            periods=[6],
            exclude_cols=["foto_mes","numero_de_cliente","target","label","weight","clase_ternaria"]
        ),
        X_transformed,
        "PeriodStatsTransformer",
        logger
    )
    new_cols = list(set(X_transformed.columns) - set(initial_cols))
    X_transformed = apply_transformer(
        TendencyTransformer(
            exclude_cols=["foto_mes","numero_de_cliente","target","label","weight","clase_ternaria"] + new_cols
        ),
        X_transformed,
        "TendencyTransformer",
        logger
    )
    new_cols = list(set(X_transformed.columns) - set(initial_cols))
    X_transformed = apply_transformer(
        DeltaLagTransformer(
            n_lags=2,
            exclude_cols=["foto_mes","numero_de_cliente","target","label","weight","clase_ternaria"] + new_cols
        ),
        X_transformed,
        "DeltaLagTransformer",
        logger
    )
    X_transformed = apply_transformer(
        PercentileTransformer(
            replace_original=True
        ),
        X_transformed,
        "PercentileTransformer",
        logger
    )
    return X_transformed

In [ ]:
features = get_features(df, [])
features = AddCanaritos(n_canaritos=5).fit_transform(features)

In [ ]:
import pickle
import lightgbm as lgb

model_file = "home/martin232009/buckets/b1/models/zlgbm-histfeatures_c5_gbzlgbm-histfeatures_s0.02_uFalse_537919.pkl"

In [ ]:
with open(model_file, 'rb') as f:
    model = pickle.load(f)

In [ ]:
features = features[features["foto_mes"] == 202108]
id = features["numero_de_cliente"]

probs = model.predict(features.drop(columns=["numero_de_cliente"]))
res = pd.DataFrame(id)
res["probs"] = probs
res.sort_values(by="probs", ascending=False)

In [ ]:
res.sort_values(by="probs", ascending=False)["numero_de_cliente"].head(12000).to_csv("entrega2.csv", index=False)